<a href="https://colab.research.google.com/github/kutyadog/ai_notebooks/blob/main/Camping_RAW_cj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Camping assistant
Early tests for building an AI assistant that can help you find campgrounds, available open camping sites, camping information,etc.

Built Jan 2024

# Start It up

In [ ]:
!pip install -q openai
!pip install tiktoken
!pip install -q gradio
!pip install wikipedia
!pip install duckduckgo_search==4.2
!pip install python-weather

In [ ]:
import openai
import tiktoken  # for counting tokens
import gradio as gr
import wikipedia
import requests
import json
from google.colab import userdata


COMPLETIONS_MODEL = "gpt-3.5-turbo-1106" # "gpt-4-1106-preview" # "gpt-3.5-turbo" #
openai.api_key = userdata.get('OPENAI_API_KEY')

def num_tokens(text: str, model: str = COMPLETIONS_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# Ai Main

In [ ]:
# @title AI Response structure

# Below are a COMPLETE breakdown of the skills and services you offer.

# 1. Where is a travel destination?
#   * Call function search_wiki

# 1. search_wiki : Search wikipedia API for Longitude and Latitude, Search For Summary of Something.
#         To call: Set 'search_wiki' value in chat_response_object function to set the query
#         :param query: str, the search query.
#         :return: wikiObject with wikiObject.coordinates for lat/longitude, wikiObject.summary for summary, wikiObject.title for title, wikiObject.content for larger content

#   Use function chat_response_object for every communication.
# Conversational text directed towards user that user needs to read should be set as value for 'chat_response' in function chat_response_object.

chat_functions = [
  {
    'name': 'get_nearby_campgrounds',
    'description': 'Get a list of nearby campgrounds to a specific location. Only use if user specifically asks for this information and you dont already have that information.',
    'parameters': {
        'type': 'object',
        'properties': {
            'location_to_query': {
                'type': 'string',
                'description': 'This is the actual location to search around. For example: "Pikes Peak"'
            },
        },
        "required": [
            "location_to_query"
        ]
    }
  },
  {
    'name': 'get_info_about_destination',
    'description': 'Get information about a specific campground or destination. Only use if user specifically asks for more information about a specific campground or destination and you dont already have that information.',
    'parameters': {
        'type': 'object',
        'properties': {
            'location_to_query': {
                'type': 'string',
                'description': 'This is the actual location to get information about. For example: "Pikes Peak"'
            },
        },
        "required": [
            "location_to_query"
        ]
    }
  },
]


introSystemText = """
You are a friendly Camping / RV travel chatbot assistant.
DO NOT ANSWER ANY QUESTIONS that do not specifically and directly answer one of the following:
A. 'What campgrounds and RV parks are near a destination?'
B. 'Tell me about campground or destination?'
C. 'Tell me what you can help me with?'
    * Tell user what services you can perform.
D. 'Who am I?'
    * Feel free to tell them all the information you have on file about them.
When the user asks for information above:
1. ALWAYS ALWAYS ALWAYS first check previous "system" role messages to make sure the answer is not already available to you without calling the function.
2. If the answer is not available in the previous "system" role messages, use the 'get_nearby_campgrounds' & 'get_info_about_destination' functions available to you.
IMPORTANT: Always remember to use previous messages for context to answer questions when capable.
"""
# IMPORTANT: Before calling any functions. Make sure the answers to the users query are not already available in previous messages.
# Only use functions when processing a "user" role message with a direct query.
# If the last message is a "system" role with data, ONLY summarize the information and communicate that to the user. Do not initiate any further function calls in your response.

# You have the following tools available to you. To use any of them, use chat_response_object function as defined for each tool:
#    1. campsites_near : Returns campgrounds nearby a destination
#         To call: Set 'campsites_near' value in chat_response_object function to the destination
#         :param query: str, the destination to search for nearby campgrounds. Example: 'Pikes Peak'
#         :return: An array of campground objects like this:
#             [{"LongName":"Cripple Creek KOA","latitude":"38.77","longitude":"-105.11","distance":"6.0"},{"LongName":"Red River KOA","latitude":"38.91","longitude":"-104.98","distance"
#    2. get_basic_info : Returns information about a campground or destination
#         To call: Set 'get_basic_info' value in chat_response_object function to the destination
#         :param query: str, the destination to search for nearby campgrounds. Example: 'Pikes Peak'
#         :return: An array of site links about the object in question, such as this:
#         {'title': 'Home | Lone Duck Campground and Cabins', 'href': 'https://www.loneduckcamp.com/', 'body': 'BOOK NOW! Our Locati of Pikes P.'}

# Only consider calling the above functions when processing a "user" role message with a direct query.
# If the message is a "system" role with data, the Assistant should only format and relay that data to the user, answering the question from their last message.


# After calling one of the functions above, the info you requested will be sent to you in a system message. When this happens, answer the users request, but DO NOT call a new function when you answer that request.
# If you have any questions or if any of this is not clear, ask them RIGHT NOW!

# When user gives new info that should be remembered, add it to summary_data in the chat_response_object.

# introSystemText = """
# Answer in the style of a friendly chatbot assistant.
# You have the following functions tools available to you:
#    1. search_wiki : Search wikipedia API for Longitude and Latitude, Search For Summary of Something.
#         :param query: str, the search query.
#         :return: wikiObject with wikiObject.coordinates for lat/longitude, wikiObject.summary for summary, wikiObject.title for title, wikiObject.content for larger content
# To call any of the above function tools, simply use function chat_response_object.
# """

messages = [
  {"role": "system", "content": introSystemText},
  {"role": "system", "content": "You are conversing with Chris Johnson. He travels with Shelby, Lincoln and Fenna. They travel with a Ford F-150 pulling a 29 foot travel trailer (Grand Design Imagine 2400BH). They have a dog and a cat."},
  {"role": "assistant", "content": "How can I help?"},
  # {'role': 'system', 'content': "Campgrounds near New Orleans: {'LongName': 'French Quarter RV Resort', 'latitude': '29.9621', 'longitude': '-90.0736', 'distance': '1.006'}, {'LongName': 'Pontchartrain Landing', 'latitude': '30.0252', 'longitude': '-90.0355', 'distance': '4.253'}, {'LongName': 'Mardi Gras RV Park & Campground', 'latitude': '30.0090', 'longitude': '-90.0178', 'distance': '4.277'}, {'LongName': 'Riverboat Travel Park', 'latitude': '30.0095', 'longitude': '-90.0148', 'distance': '4.447'}, {'LongName': 'Jude Travel Park', 'latitude': '30.0112', 'longitude': '-90.0003', 'distance': '5.261'}, {'LongName': '3 Oaks & A Pine RV Park', 'latitude': '30.0121', 'longitude': '-89.9996', 'distance': '5.327'}, {'LongName': 'Parc D’Orleans RV Park', 'latitude': '30.0118', 'longitude': '-89.9975', 'distance': '5.429'}, {'LongName': 'Marrero Travel Park', 'latitude': '29.9006', 'longitude': '-90.1069', 'distance': '5.493'}, {'LongName': 'Bayou Segnette State Park Campground', 'latitude': '29.9029', 'longitude': '-90.1541', 'distance': '6.795'}, {'LongName': 'New Orleans West KOA', 'latitude': '29.9702', 'longitude': '-90.2348', 'distance': '9.377'}, {'LongName': 'St. Bernard State Park Campground', 'latitude': '29.8644', 'longitude': '-89.9004', 'distance': '13.156'}, {'LongName': 'D & D RV Park', 'latitude': '29.9017', 'longitude': '-90.3858', 'distance': '19.117'}, {'LongName': 'LaPlace Trailer Park & RV Park', 'latitude': '30.0540', 'longitude': '-90.4618', 'distance': '23.568'}"}
  # {"role": "user", "content": message},
]

# ------------------------------------- interface
def ask_question(question, model):
  return ask(query=question, model=model).replace("```html", "").replace("```", "")

def respond(message, chat_history: list, modelName):
  bot_message = ask_question(message, modelName)
  messages.append({"role": "assistant", "content": bot_message})
  return bot_message


In [ ]:
# @title Analyze Ai Response Processing

def analyzeAiResponse(response) :
    print('------------------------------------AI response')
    print(response)

    if (response.choices[0].message.function_call == None ) :

      response_message = response.choices[0].message.content
      # print("response message 1: " + response_message )
      return ai_response(response_message)

    else:
      print(response.choices[0].message.function_call)
      func_name                   = response.choices[0].message.function_call.name
      response_json               = json.loads(response.choices[0].message.function_call.arguments)

      # FunctionCall(arguments='{"location_to_query":"Mt Hood"}', name='get_nearby_campgrounds')

      print("func_name: " + func_name )

      if ( func_name == 'get_nearby_campgrounds') :
        location_to_query    = getJsonValue(response_json, 'location_to_query')
        print("get_nearby_campgrounds: " + location_to_query)
        wiki_response = search_wiki(location_to_query)
        if wiki_response == False :
          return ai_response("Sorry, having troubles finding that location (wiki).")

        print( 'wiki coordinates: ' + str(wiki_response.coordinates[0]) + ', '+ str(wiki_response.coordinates[1]) )

        campgrounds = get_nearby_campgrounds(wiki_response.coordinates[0], wiki_response.coordinates[1])
        print("Retrieved campground information")
        # print(type(campgrounds))

        messages.append(
          {
              "role": "assistant",
              "function_call": {
                  "name": 'get_nearby_campgrounds',
                  "arguments": location_to_query,
              },
              "content": None
          }
        )
        messages.append( # adding function response to messages
            {
                "role": "function",
                "name": 'get_nearby_campgrounds',
                "content": str(', '.join([str(x) for x in campgrounds]) ),
            }
        )

        # messages.append({"role": "system", "content": 'Campgrounds near '+ location_to_query +': ' + str(', '.join([str(x) for x in campgrounds]) ) })
        messages.append({"role": "system", "content": 'Summarize info from previous system message. DO NOT use functions in this response!' })

        # messages.append({"role": "system", "content": 'Here is the answer to nearby campgrounds: ' + str(', '.join([str(x) for x in campgrounds]) ) })
        # messages.append({"role": "system", "content": 'Your next response should tell the user about the campsites, WITHOUT CALLING any new functions!'})
        # messages.append({"role": "system", "content": 'Provide the user with the following campsite information. Do not initiate any further function calls in this response: ' + str(', '.join([str(x) for x in campgrounds]) ) })
        # messages.append({"role": "system", "content": 'Your next response should tell the user about the campsites, WITHOUT CALLING any new functions!'})
        # print( " ** Telling AI to INFORM")
        # for campground in messages:
        #   print(campground["role"] + ': '+ campground["content"])

        return ask( '' )

      elif ( func_name == 'get_info_about_destination') :
        location_to_query    = getJsonValue(response_json, 'location_to_query')
        print(" - - Getting info about: " + location_to_query)


        search = DDGS()
        search_results = search.text(location_to_query, max_results=5)
        print(" - - Info retrieved - Sending info to AI to summarize")
        dataList = []
        for result in search_results:
            # Process each search result
            # print(result)
            dataList.append(result)

        json_string = json.dumps(dataList)
        # print(json_string)
        # messages.append({"role": "assistant", "content": bot_message})
        messages.append(
          {
              "role": "assistant",
              "function_call": {
                  "name": 'get_info_about_destination',
                  "arguments": location_to_query,
              },
              "content": None
          }
        )
        messages.append( # adding function response to messages
            {
                "role": "function",
                "name": 'get_info_about_destination',
                "content": json_string,
            }
        )

        # messages.append({"role": "system", "content": 'Info for '+ location_to_query +': ' + json_string })
        messages.append({"role": "system", "content": 'Summarize info from previous system message. DO NOT initiate any further function calls in your response!' })

        # messages.append({"role": "system", "content": 'Your next response should tell the user about '+ get_basic_info_string +' based on info in previous message, without calling any new functions.'})
        # print( " ** Telling AI to tell user now about "+ location_to_query)

        # for campground in messages:
        #   print(campground["role"] + ': '+ campground["content"])

        return ask( '' )

    return ai_response(response_message)


In [ ]:
# @title Asking AI

def getJsonValue( json_object, keyname ) :
  if keyname in json_object:
    return json_object[keyname]
  else :
    return ''

def ai_response( responseString : str ) :
  messages.append({"role": "assistant", "content": responseString})
  return responseString

def ask(
    query: str,
    model = COMPLETIONS_MODEL,
    token_budget: int = 4096 - 500,
    print_message: bool = False,
) -> str:
    global COMPLETIONS_MODEL
    print("------------------------------------------Asking AI New Question")
    print(query)
    # print("chat functions:")
    # print(chat_functions)
    COMPLETIONS_MODEL = model # must keep this here if we want user to be able to change models

    if query != '' :
      messages.append({"role": "user", "content": query})

    print('messages length: ' + str(len(messages)) )
    # print(messages)

    response = openai.chat.completions.create(
        model=COMPLETIONS_MODEL,
        messages=messages,
        functions = chat_functions,
        temperature=0,
    )

    return analyzeAiResponse(response)




In [ ]:
# @title Asking ----- AI TESTING

# ask_question( "What campgrounds are near New Orleans?", COMPLETIONS_MODEL)
# ask_question( "What can you tell me about Mount hood in oregon?", COMPLETIONS_MODEL)
# ask_question( "Sounds nice. Can you tell me which campgrounds are near there?", COMPLETIONS_MODEL)
# ask_question( "Awesome. How far is the Toll Bridge one from there?", COMPLETIONS_MODEL)
# ask_question( "What is the elevation for that campsite?", COMPLETIONS_MODEL)
# ask_question( "Sorry, i forgot to write down which campgrounds were near Mount Hood? Can you tell them to me again?", COMPLETIONS_MODEL)
# ask_question( "What can you tell me about Pontchartrain Landing?", COMPLETIONS_MODEL)
# ask_question( "Tell me again about Pontchartrain Landing?", COMPLETIONS_MODEL)
# ask_question( "What two colors make purple?", COMPLETIONS_MODEL)
# ask_question( "How can you help me?", COMPLETIONS_MODEL)
ask_question( "which campgrounds are near Mt Hood again?", COMPLETIONS_MODEL)
# ask_question( "tell me about Camp Creek", COMPLETIONS_MODEL)
# ask_question( "what were the others besides Camp Creek", COMPLETIONS_MODEL)
# ask_question( "What do you know about me?", COMPLETIONS_MODEL)
# ask_question( "Why did you use the get_info_about_destination destination for Pontchartrain Landing when you already had the information in the conversation history? How can I modify your prompts to force you to look at your conversation history before calling a new function?", COMPLETIONS_MODEL)

In [ ]:
messages

# **Ai Functions**

In [ ]:
# @title Weather

import python_weather

# https://github.com/null8626/python-weather

async def getweather(location):
  # location = 'Louisville'
  # async def getweather(location):
  #   # declare the client. the measuring unit used defaults to the metric system (celcius, km/h, etc.)
  async with python_weather.Client(unit=python_weather.IMPERIAL) as client:
    # fetch a weather forecast from a city
    weather = await client.get(location)
    print(str(weather))
    print(str(weather.location ))
    print(str(weather.location[0] ))
    print(str(weather.location[1] ))
    print(str(weather.current))
    print(str(weather.current.date ))
    print(str(weather.nearest_area))
    # returns the current day's forecast temperature (int)
    print(weather.current.temperature)
    print('----------')
    print(weather.current.description)
    print('----------')
    print(weather.current.wind_speed)
    print('----------')
    print(weather.current.visibility)
    print('----------')
    print(weather.current.precipitation)
    print('----------')
    print(weather.current.humidity)
    print('----------')
    print(weather.nearest_area.name )
    print('----------')
    print(weather.nearest_area.region )
    print('----------')
    print(weather.nearest_area.latitude )
    print(weather.nearest_area.longitude )
    return weather

    # get the weather forecast for a few days
    # for forecast in weather.forecasts:
    #   print(forecast)

    #   # hourly forecasts
    #   for hourly in forecast.hourly:
    #     print(f' --> {hourly!r}')

await getweather("Mount Hood")



In [ ]:
# @title wiki functions

# wiki_page = wikipedia.page(page, **load_kwargs)
# page_content = wiki_page.content
# page_url = wiki_page.url

# print( wikipedia.search("Crater Lake"))
# ['Crater Lake', 'Volcanic crater lake', 'Lonar Lake', 'Crater Lake National Park', 'Crater Lake (disambiguation)', 'Taal Volcano Main Crater Lake', 'Impact crater lake', 'Jezero (crater)', 'Manicouagan Reservoir', 'Taal Volcano']

# wiki_response = wikipedia.page(title="Crater Lake", auto_suggest=False)
# print( 'wiki coordinates: ' + str(wiki_response.coordinates[0]) + ', '+ str(wiki_response.coordinates[1]) )
import requests


def search_wiki(query):
    print( 'search_wiki: '+ query )
    """
    Search For Longitude and Latitude
    Search For Summary of Something
    Search Internet Tool
    search_wiki tool
    Performs a search on wikipedia using the wikipedia API.

    :param query: str, the search query.
    :return: wikiObject with wikiObject.coordinates for lat/longitude, wikiObject.summary for summary, wikiObject.title for title, wikiObject.content for larger content
    """
    try:
      wikiObject = wikipedia.page(title=query, auto_suggest=False)
    except:
      print("HAVCING ROUBLE WITH WIKI")
      # Code to handle the exception
      return False
      # wikiObject = { 'chat_response' : "Sorry I am having trouble finding that (wiki)."}

    return wikiObject

# Example usage:
# results = search_wiki('Crater Lake')



def wiki_open_search( query, limit = 5 ) :
  S = requests.Session()

  URL = "https://en.wikipedia.org/w/api.php"

  PARAMS = {
      "action": "opensearch",
      "namespace": "0",
      "search": query,
      "limit": limit,
      "format": "json"
  }

  R = S.get(url=URL, params=PARAMS)
  DATA = R.json()
  # print(DATA)
  # print(DATA[1][0])

  if ( len(DATA[1]) == 0 ) :
    return ''

  return DATA[1][0]


# ------------------------------------THIS DOES NOT WORK FULLY
def wiki_coordinates( query ) :
  S = requests.Session()

  URL = "https://en.wikipedia.org/w/api.php"

  PARAMS = {
      "action": "query",
      "format": "json",
      "titles": query,
      "prop": "coordinates",
  }

  R = S.get(url=URL, params=PARAMS)
  DATA = R.json()
  PAGES = DATA['query']['pages']
  print(DATA)

  try:
    returnString = ''
    for k, v in PAGES.items():
        returnString += "Latitute: " + str(v['coordinates'][0]['lat'])
        returnString += ", Longitude: " + str(v['coordinates'][0]['lon'])
    return returnString
  except:
    return ''

  return ''


# print( wiki_coordinates( "Mount Hood, oregon" ) )


# https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch=pikes%20peak&format=json

# https://www.campsitebadger.com/api.php?xtype=100&lat=38.8405322&lng=-105.0442&dist=25

# https://en.wikipedia.org/w/api.php?action=opensearch&search=pikes%20peak&format=json

# ***************
#!/usr/bin/python3

# """
#     opensearch.py
#     MediaWiki API Demos
#     Demo of `Opensearch` module: Search the wiki and obtain
# 	results in an OpenSearch (http://www.opensearch.org) format
#     MIT License
# """

# import requests

# S = requests.Session()

# URL = "https://en.wikipedia.org/w/api.php"

# PARAMS = {
#     "action": "opensearch",
#     "namespace": "0",
#     "search": "Hampi",
#     "limit": "5",
#     "format": "json"
# }

# R = S.get(url=URL, params=PARAMS)
# DATA = R.json()

# print(DATA)

# ***************

# https://en.wikipedia.org/w/api.php?action=query&generator=geosearch&prop=coordinates%7Cpageimages&ggscoord=37.7891838%7C-122.403352&format=json

# """
#     geocoordinates.py

#     MediaWiki API Demos
#     Demo of `Geosearch` module: Obtain coordinates for wiki pages nearby

#     MIT License
# """

# import requests

# S = requests.Session()

# URL = "https://en.wikipedia.org/w/api.php"

# PARAMS = {
#     "action": "query",
#     "format": "json",
#     "titles": "Wikimedia Foundation",
#     "prop": "coordinates"
# }

# R = S.get(url=URL, params=PARAMS)
# DATA = R.json()
# PAGES = DATA['query']['pages']

# for k, v in PAGES.items():
#     print("Latitute: " + str(v['coordinates'][0]['lat']))
#     print("Longitude: " + str(v['coordinates'][0]['lon']))



In [ ]:
# @title test wiki

# print( wikipedia.search("downtown New Orleans"))
# ['Crater Lake', 'Volcanic crater lake', 'Lonar Lake', 'Crater Lake National Park', 'Crater Lake (disambiguation)', 'Taal Volcano Main Crater Lake', 'Impact crater lake', 'Jezero (crater)', 'Manicouagan Reservoir', 'Taal Volcano']


# try:
#   wiki_response = wikipedia.page(title="Downtown New Orleans", auto_suggest=False)
#   print( 'wiki coordinates: ' + str(wiki_response.coordinates[0]) + ', '+ str(wiki_response.coordinates[1]) )
# except:
#   print('oops')

# try:
#   wiki_response = wikipedia.search("downtown new orleans")
#   print( wiki_response )
# except:
#   print('oops')

print(wikipedia.page(title="Downtown New Orleans", auto_suggest=False) )






<WikipediaPage 'Downtown New Orleans'>


In [ ]:
# @title Get Nearby Campgrounds

def get_nearby_campgrounds(latitude, longitude):
  """
  This function takes in a latitude and longitude and returns a list of nearby campgrounds.

  Args:
    latitude: The latitude of the user's location.
    longitude: The longitude of the user's location.

  Returns:
    A list of dictionaries containing information about nearby campgrounds.
  """

  # api.php?xtype=100&lat=38.8405322&lng=-105.0442&dist=25
  base_url = userdata.get('full_api_url') + "/api.php"
  params = {"xtype": "100", "lat": latitude, "lng": longitude, "dist": "25" }

  response = requests.get(base_url, params=params)

  if response.status_code == 200:
    campgrounds = response.json()
    return campgrounds
  else:
    raise Exception("Error fetching campgrounds")

# # Example usage
# latitude = "38.77471160"
# longitude = "-105.11689000"

# campgrounds = get_nearby_campgrounds(latitude, longitude)

# for campground in campgrounds:
#   print(campground["LongName"])

# [{"xnum":"5755","LongName":"Cripple Creek KOA","latitude":"38.77471160","longitude":"-105.11689000","object_id":"3305","marker_icon":"2","distance":"6.000318613805682"}]


In [ ]:
# @title Duck Duck Go

import duckduckgo_search
from duckduckgo_search import DDGS

def search_duckGoDuck(query, max_results=1):
    """
    Search Skill
    Search Tool
    Search Internet Tool
    Search Internet Skill
    duckduckgo_search tool
    Performs a search on DuckDuckGo using the duckduckgo_search package.

    :param query: str, the search query.
    :param max_results: int, maximum number of results to return.
    :return: list, search results.
    """
    with DDGS() as ddgs:
        return [r for r in ddgs.text(query, max_results=max_results)]

# Example usage:
# results = search_internet('latitude longitude of pikes peak')
# print(results)


In [ ]:
# @title test Duck DUck Go

# results = search_duckGoDuck('fire')
# print(results)
import json

search = DDGS()
search_results = search.text('Lone Duck Family Campground', max_results=5)
# print(type(search_results) )
# Process the search results as needed
dataList = []
for result in search_results:
    # Process each search result
    print(result)
    dataList.append(result)

json_string = json.dumps(dataList)
print(json_string)

# Gradio Interface

In [ ]:
# @title Gradio Interface

with gr.Blocks() as demo:
    slider = gr.Radio(["gpt-4-1106-preview", "gpt-3.5-turbo", "gpt-3.5-turbo-1106"], value=COMPLETIONS_MODEL, label="Model", interactive=True)
    chat = gr.ChatInterface(respond, retry_btn=None, undo_btn=None, additional_inputs=slider )

demo.launch(debug=True)
# demo.launch()


In [ ]:
# @title colab mysql

# install, set connection
!apt-get install mysql-server > /dev/null
!service mysql start
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'"
!pip -q install PyMySQL
%load_ext sql
%config SqlMagic.feedback=False
%config SqlMagic.autopandas=True
%sql mysql+pymysql://root:root@/
# query using %sql or %%sql
df = %sql SELECT Host, User, authentication_string FROM mysql.user
df

# GOOGLE API

In [ ]:
from google.colab import userdata


# -------------------------------------------THIS IS THE REAL GOOGLE API
# https://customsearch.googleapis.com/customsearch/v1?cx=54eef59c8da97416f&q=mount%20hood&key={userdata.get('GOOGLE_API_KEY_3')}
# https://customsearch.googleapis.com/customsearch/v1?cx=54eef59c8da97416f&q=fast%20cat&key={userdata.get('GOOGLE_API_KEY_3')}

# https://github.com/Nv7-GitHub/googlesearch/blob/master/googlesearch/__init__.py

# https://github.com/googleapis/google-api-python-client/tree/main/samples

# cloud api mysql instance
# camping-api-test
# eatPickle44
# chrisj
# eatsPicklesOnTuesday5

# ----------- this is not an official google release, it basically scrapes google
# !pip install googlesearch-python

# from googlesearch import search
# searchResults = search("Mt Hood", advanced=True, num_results=10)

# for item in searchResults:
#   print(item.title)
#   print(item.url)
#   print(item.description)

# # Returns a list of SearchResult
# # Properties:
# # - title
# # - url
# # - description



# Php remote connector

In [ ]:
!pip install mysql-connector-python

In [ ]:
import mysql.connector
from google.colab import userdata

connection = mysql.connector.connect(
               host=userdata.get('CJNET_MYSQL_IP'),
               database=userdata.get('CJNET_MYSQL_DB'),
               user=userdata.get('CJNET_MYSQL_USER'),
               password=userdata.get('CJNET_MYSQL_PWD'))

print(connection)

cursor = connection.cursor()

cursor.execute('SELECT * FROM campgrounds_big_2024 WHERE 1')

results = cursor.fetchall()

for row in results:
    print(row)

cursor.close()
connection.close()

In [ ]:
!curl ipecho.net/plain

In [ ]:
!pip install PyMySQL

In [ ]:
# @title remote mysql, not working

import pymysql.cursors

# Connect to the database
connection = pymysql.connect(
               host=userdata.get('CJNET_MYSQL_IP'),
               database=userdata.get('CJNET_MYSQL_DB'),
               user=userdata.get('CJNET_MYSQL_USER'),
               password=userdata.get('CJNET_MYSQL_PWD'),
                             cursorclass=pymysql.cursors.DictCursor)

with connection:
    with connection.cursor() as cursor:
        # Create a new record
        sql = "SELECT * FROM campgrounds_big_2024 WHERE 1"
        cursor.execute(sql)
    # connection is not autocommit by default. So you must commit to save
    # your changes.
    connection.commit()

    with connection.cursor() as cursor:
        # Read a single record
        sql = "SELECT * FROM campgrounds_big_2024 WHERE 1"
        cursor.execute(sql)
        result = cursor.fetchone()
        print(result)

# Spacy - intent detection tests
So far no good

In [ ]:
# @title Intent Detection

import spacy

# Load the pre-trained English model
nlp = spacy.load('en_core_web_sm')

# Intent detection function
def detect_intent(text):
    doc = nlp(text)
    # Assume NLP model or predefined logic exists for determining the intent
    # For demonstration, we return a mock intent based on the text
    # return 'book_flight' if 'flight' in text.lower() else 'unknown'
    return doc

# Use detect_intent function within the chatbot workflow
user_input = "ahh can you tell me what camgrounds are around Pikes Peak?"
intent = detect_intent(user_input)
# if intent == 'book_flight':
#     response = handle_booking()
# else:
#     response = "I'm not sure how to help with that."

print(intent)

In [ ]:
!pip install spacy

# Older Simple Chat

In [ ]:



def display_chat_history(messages):
    for message in messages:
      if message['role'] != "system":
        print(f"{message['role'].capitalize()}: {message['content']}")


def get_assistant_response(messages):
    r = openai.chat.completions.create(
        model="gpt-3.5-turbo-0613", #"gpt-3.5-turbo",
        messages=[{"role": m["role"], "content": m["content"]} for m in messages],
        functions = chat_functions,
        temperature=0,
    )

    print('------------AI response')
    print(r)
    response = r.choices[0].message.content
    return response

    # print('------------AI response')
    # print(response)
    # response_json = json.loads(response.choices[0].message.function_call.arguments)
    # print(response_json)
    # response_keywords = response_json['keywords'].split(', ')
    # response_meta_descriptions = response_json['meta_description_examples'].split('**')
    # response_meta_titles = response_json['meta_titles'].split('**')
    # print("meta 1")
    # response_json['keywords'] = [s.strip() for s in response_keywords]
    # response_json['meta_descriptions'] = [s.strip() for s in response_meta_descriptions][:3]
    # response_json['meta_titles'] = [s.strip() for s in response_meta_titles][:3]


# messages = [{"role": "assistant", "content": "How can I help?"}]

introSystemText = """
Answer in the style of a friendly chatbot assistant.
You have the following functions tools available to you:
   1. search_wiki : Search wikipedia API for Longitude and Latitude, Search For Summary of Something.
        :param query: str, the search query.
        :return: wikiObject with wikiObject.coordinates for lat/longitude, wikiObject.summary for summary, wikiObject.title for title, wikiObject.content for larger content
To call any of the above function tools, simply,
Always use function chat_response_object, inserting any dialog for the user in 'chat_response' property.
"""

messages = [
  {"role": "system", "content": introSystemText},
  {"role": "assistant", "content": "How can I help?"}
  # {"role": "user", "content": message},
]

# ChatCompletion(id='chatcmpl-8lOlXorJ263FxeSf2X7Ruo59yarOQ', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(
#     content=None, role='assistant',
#     function_call=FunctionCall(arguments='{\n  "chat_response": "Pikes Peak is located in the Rocky Mountains of Colorado, United States. It is one of the most famous mountains in the state."\n}', name='chat_response_object'), tool_calls=None))], created=1706307307, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=42, prompt_tokens=276, total_tokens=318))

while True:
    display_chat_history(messages)

    user_input = input("User: ")
    messages.append({"role": "user", "content": user_input})

    assistant_response = get_assistant_response(messages)
    messages.append({"role": "assistant", "content": assistant_response})

In [ ]:
# @title ChatGPT answer for finding intent quickly
def handle_greeting():
    return "Hello! How can I help you?"

def handle_booking():
    return "Let's book something for you."

def handle_farewell():
    return "Goodbye! Have a great day."

# Map keywords to their corresponding functions
keywords = {
    'hello': handle_greeting,
    'hi': handle_greeting,
    'book': handle_booking,
    'reserve': handle_booking,
    'bye': handle_farewell,
    'goodbye': handle_farewell,
}

def determine_goal(user_input):
    for keyword, function in keywords.items():
        if keyword in user_input.lower():
            return function()
    return "I'm not sure how to help with that."

# Example usage
user_input = "I would like to book a flight."
response = determine_goal(user_input)
print(response)




# -------------------------------

import spacy

# Load the pre-trained English model
nlp = spacy.load('en_core_web_sm')

# Intent detection function
def detect_intent(text):
    doc = nlp(text)
    # Assume NLP model or predefined logic exists for determining the intent
    # For demonstration, we return a mock intent based on the text
    return 'book_flight' if 'flight' in text.lower() else 'unknown'

# Use detect_intent function within the chatbot workflow
user_input = "Please help me with booking a flight."
intent = detect_intent(user_input)
if intent == 'book_flight':
    response = handle_booking()
else:
    response = "I'm not sure how to help with that."

print(response)

# Future Features:
Below are resources I could possibly add to project down the road.

Places to stay for free like Walmart,


Lots of free APIs
https://rapidapi.com/category/Travel


https://github.com/public-apis/public-apis
https://github.com/public-apis/public-apis#weather



https://www.allstays.com/c/truck-stop-locations.htm


Casinos - Some casinos allow free overnight parking for motorhomes. A good resource to locate them is Casino Camper.
Truck Stops - Many truck stops will let you stay overnight for free. To locate these see Truck Stop Locations.
overnightrvparking.com - Another resource for free overnight parking is overnightrvparking.com. They also offer a mobile app for your convenience.
boondockerswelcome.com - Free overnight parking on private property.
Harvest Hosts - Free overnight camping at wineries, farms, and agri-tourism sites.

LIST LOCATIONS OF EV VEHICLE CHARGERS

Recreation Information Database - RIDB
HIKING, CAMPING, BEACHES, ETC. FREE
https://ridb.recreation.gov/landing



In [ ]:
from google.colab import userdata


# https://rapidapi.com/MyAllies/api/breaking-news/

# this requires payment
def rapid_api_trails( latitude, longitude ) :

  URL = "https://trailapi-trailapi.p.rapidapi.com/trails/explore/"

  S = requests.Session()
  S.headers['X-RapidAPI-Key'] = userdata.get('RAPIDAPI_KEY')
  S.headers['X-RapidAPI-Host'] = userdata.get('RAPIDAPI_HOST')

  PARAMS = {
      "lat": latitude,
      "lon": longitude,
  }

  R = S.get(url=URL, params=PARAMS)
  DATA = R.json()
  print(DATA)

rapid_api_trails(  45.3736111099999988027775543741881847381591796875, -121.695 )